In [ ]:
import sys
from helper import *
from datetime import datetime, timedelta
from indicators import SslChannel, ExponentialMovingAverage
from strategies import SslChannelEmaStrategy
import mplfinance as mpf
from api.api import *

In [ ]:
ssl_channel_ema_strategy = SslChannelEmaStrategy()

In [ ]:
candlestick_5m, candlestick_1h = ssl_channel_ema_strategy.calculate_current_trend()

In [ ]:
ssl_channel = SslChannel()
ema = ExponentialMovingAverage()

In [ ]:
candlestick_df_ssl = ssl_channel.apply_indicator(candlestick_5m)
candlestick_df_ssl_ema = ema.apply_indicator(candlestick_df_ssl)

In [ ]:
candlestick_df_ssl_ema.tail()

In [ ]:
# Plots
apds = [
    mpf.make_addplot(candlestick_df_ssl_ema[['ssl_channel_ssl_down']], color='violet'),
    mpf.make_addplot(candlestick_df_ssl_ema[['ssl_channel_ssl_up']], color='blue'),
    mpf.make_addplot(candlestick_df_ssl_ema[['ema_10']], color='orange'),
    mpf.make_addplot(candlestick_df_ssl_ema[['ssl_channel_trend']])
]


mpf.plot(candlestick_df_ssl_ema, type='candle', style='charles', addplot=apds)

In [ ]:
from connectors import BitstampConnector,DummyConnector

In [ ]:
bitstamp_connector = BitstampConnector()
dummy_connector =DummyConnector()

In [ ]:
bitstamp_connector.get_balance()

In [ ]:
dummy_connector.get_balance()

In [ ]:
dummy_connector.buy_eth(0.03, 1000)

In [ ]:
dummy_connector.get_balance()

In [ ]:
dummy_connector.sell_eth(0.05, 1000)
dummy_connector.get_balance()

In [ ]:
bitstamp_connector.buy_eth(0.05, 1000)

In [ ]:
bitstamp_connector.sell_eth(0.04, 10000)

In [ ]:
# Initial account-balance
from models import InfluxConnector, AccountBalance
from datetime import datetime

influx_connector = InfluxConnector()
client = influx_connector.client

current_account_balance = AccountBalance(timestamp_utc=datetime.utcnow(), pair="ETH-EUR", exchange="Bitstamp", eth_available=0, eur_available=500, balance_total=500)
influx_connector.write_point(current_account_balance.to_influx(connector="simulator"))


In [ ]:
from connectors import BitstampConnector,DummyConnector

dummy_connector = DummyConnector()
current_balance = dummy_connector.get_balance()

print("current_balance.eur_available",current_balance.eur_available)


In [ ]:
dummy_connector.buy_eth(0.05,1000)

In [ ]:
dummy_connector.sell_eth(0.03,1050)

In [ ]:
dummy_connector.sell_eth(0.02,1100)

In [ ]:
from connectors import DummyConnector
from strategies import SimpleStrategy

dummy_connector = DummyConnector()
simple_strategy = SimpleStrategy()


In [ ]:
from api.api import get_eth_eur_values
from helper import create_candlesticks
import mplfinance as mpf
live_trades= get_eth_eur_values(from_dt_str="now()-4d")
candlestick_5m = create_candlesticks(live_trades)

In [ ]:
candlestick_5m.tail()

In [ ]:
candlestick_5m= candlestick_5m.set_index('timestamp_utc')

In [ ]:
mpf.plot(candlestick_5m, type='candle', style='charles')

In [ ]:
import talib


In [ ]:
candle_names = talib.get_function_groups()['Pattern Recognition']

In [ ]:
candlestick_5m['engulfing'] = talib.CDLENGULFING(candlestick_5m['open'],candlestick_5m['high'], candlestick_5m['low'], candlestick_5m['close'])

In [ ]:
candlestick_5m.loc[candlestick_5m.engulfing>0]

In [ ]:
#candlestick_5m.tail(50)

In [ ]:
candlestick_5m['ema_10'] = talib.EMA( candlestick_5m['close'],10)
candlestick_5m['ema_20'] = talib.EMA( candlestick_5m['close'],20)

In [ ]:
candlestick_5m.tail(50)

In [ ]:
# Plots
apds = [
    mpf.make_addplot(candlestick_5m[['ema_10']], color='violet'),
    mpf.make_addplot(candlestick_5m[['ema_20']], color='orange'),
]
mpf.plot(candlestick_5m, type='candle', style='charles', addplot=apds)

In [ ]:
from connectors import BitstampConnector,DummyConnector

dummy_connector = DummyConnector()
current_balance = dummy_connector.get_last_transaction(type='sell')
print("current_balance.amount",current_balance.amount)

bitstamp_connector = BitstampConnector()
last_transaction= dummy_connector.get_last_transaction()
if last_transaction.id is None:
    print("empty transactions")
else:
    print("last_transaction.id",last_transaction.id)

In [21]:
from connectors import DummyConnector
from strategies import SimpleStrategy

dummy_connector = DummyConnector()
simple_strategy = SimpleStrategy()

influx_client <influxdb.client.InfluxDBClient object at 0x7f1a3c388650>
type(result_points[idx]) <class 'dict'>
influx_connector InfluxConnector(client=<influxdb.client.InfluxDBClient object at 0x7f1a3c305f10>)
account_balance.to_influx() {'measurement': 'simulator_account_balance', 'tags': {'pair': 'ETH-EUR', 'exchange': 'Bitstamp'}, 'time': datetime.datetime(2021, 8, 31, 7, 58, 36, 925856), 'fields': {'eur_available': 1000.0, 'eth_available': 1.5, 'balance_total': 5253.61}}


In [22]:
data = simple_strategy.apply(dummy_connector)

influx_client <influxdb.client.InfluxDBClient object at 0x7f1a3c3056d0>
type(result_points[idx]) <class 'dict'>
live_trades 1254
influx_client <influxdb.client.InfluxDBClient object at 0x7f1a3c48cb10>
type(result_points[idx]) <class 'dict'>
current eth-eur-value: 2832.89
query_str SELECT * FROM simulator_transactions order by time desc limit 1
out-trade


In [1]:
data.tail(31)

NameError: name 'data' is not defined

In [ ]:
data.loc[data.index >"2021-08-27 20:50"]

In [ ]:
data.tail()

In [ ]:
data[:-1]

In [ ]:
last_records= data[-2:]

In [ ]:
len(last_records.loc[last_records['engulfing']])>0

In [5]:
from connectors import DummyConnector, BitstampConnector
dummy_connector = DummyConnector()
bitstamp_connector = BitstampConnector()
current_dummy_balance = dummy_connector.get_balance()
current_bitstamp_balance = bitstamp_connector.get_balance()

influx_client <influxdb.client.InfluxDBClient object at 0x7fd218c2e310>
type(result_points[idx]) <class 'dict'>
influx_connector InfluxConnector(client=<influxdb.client.InfluxDBClient object at 0x7fd218ae6610>)
account_balance.to_influx() {'measurement': 'simulator_account_balance', 'tags': {'pair': 'ETH-EUR', 'exchange': 'Bitstamp'}, 'time': datetime.datetime(2021, 8, 29, 20, 32, 30, 288162), 'fields': {'eur_available': 503.5, 'eth_available': 0.0, 'balance_total': 503.5}}
influx_client <influxdb.client.InfluxDBClient object at 0x7fd2189b9890>
type(result_points[idx]) <class 'dict'>
influx_connector InfluxConnector(client=<influxdb.client.InfluxDBClient object at 0x7fd219f93a90>)
account_balance.to_influx() {'measurement': 'bitstamp_account_balance', 'tags': {'pair': 'ETH-EUR', 'exchange': 'Bitstamp'}, 'time': datetime.datetime(2021, 8, 29, 20, 32, 30, 560265), 'fields': {'eur_available': 0.85, 'eth_available': 2.14, 'balance_total': 5882.26}}


In [6]:
dummy_connector.tradeable_eth()

self.eth_reserve 1.5


-1.5

In [7]:
bitstamp_connector.tradeable_eth()

self.eth_reserve 1.5


0.6446775800000002

In [7]:
current_bitstamp_balance.eur_available

0.85

In [12]:
from api.api import get_current_eth_eur_value
current_eth_eur_value = getattr(get_current_eth_eur_value(),'price')

influx_client <influxdb.client.InfluxDBClient object at 0x7fd21893e410>
type(result_points[idx]) <class 'dict'>


In [13]:
current_eth_eur_value

2744.46

In [23]:
last_records= data[-2:]

NameError: name 'data' is not defined

In [4]:
last_records = data[-2:-1]
last_relevant_record = data[-2:-1]
predecessor_record = data[-3:-2]

In [17]:
length_predecessor_candle = abs(predecessor_record['open'].values[0]-predecessor_record['close'].values[0])

In [20]:
last_relevant_record = data[-2:-1].values
last_relevant_record

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [16]:
0.2*length_predecessor_candle

timestamp_utc
2021-08-30 23:55:00+00:00    0.414
Name: close, dtype: float64

In [18]:
if ( last_relevant_record['close'].values[0]< predecessor_record['open'].values[0] + 0.2*length_predecessor_candle):
    print('true')
else:
    print("false")

true


In [9]:
print("last buying price",5, "current_eth_eur_value",6)

last buying price 5 current_eth_eur_value 6


In [1]:
# Initial account-balance
from models import InfluxConnector, AccountBalance
from datetime import datetime

influx_connector = InfluxConnector()
client = influx_connector.client

current_account_balance = AccountBalance(timestamp_utc=datetime.utcnow(), pair="ETH-EUR", exchange="Bitstamp", eth_available=1.5, eur_available=1000, balance_total=5247.65)
influx_connector.write_point(current_account_balance.to_influx(connector="simulator"))

In [6]:
import pandas as pd
import numpy as np
df = pd.DataFrame(np.random.randn(10,3),columns=['A','B','C'])
df = df.set_index('A')

In [12]:
df.loc[df['B'] > 0.390563]

,B,C
A,,
-1.416954,0.96938,-1.013256
-1.423276,0.95893,-0.148584


In [17]:
max(df.loc[df['B'] > 0.390563].idxmax())

-1.4169540763379864